In [ ]:
!pip install pandas numpy matplotlib seaborn scikit-learn tensorflow keras gradio google-generativeai

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import google.generativeai as genai
import gradio as gr
import io
import os
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from IPython.display import display, HTML
from pathlib import Path

# Create visualizations directory
if not os.path.exists('visualizations'):
    os.makedirs('visualizations')

"""## Configuración de la API de Gemini AI"""
def setup_gemini_api(api_key):
    try:
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel('gemini-1.5-flash')
        print("✅ API de Gemini configurada correctamente")
        return True, model
    except Exception as e:
        print(f"❌ Error al configurar la API de Gemini: {e}")
        print("Intenta listar los modelos disponibles con el siguiente código:")
        print("""
        import google.generativeai as genai
        genai.configure(api_key='YOUR_API_KEY')
        for model in genai.list_models():
            print(f"Model: {model.name}, Supported Methods: {model.supported_generation_methods}")
        """)
        return False, None

"""## Funciones para el análisis de datos"""
def generate_csv_stats(df, max_preview_rows=100):
    buffer = io.StringIO()
    buffer.write(f"### Información general del archivo\n")
    buffer.write(f"- Número de filas: {df.shape[0]}\n")
    buffer.write(f"- Número de columnas: {df.shape[1]}\n")
    buffer.write(f"- Columnas: {', '.join(df.columns.tolist())}\n\n")

    # Vista previa personalizable
    buffer.write("### Vista previa de los datos\n")
    if df.shape[0] > max_preview_rows:
        buffer.write(f"Mostrando las primeras {max_preview_rows} filas de {df.shape[0]} totales:\n")
        buffer.write(df.head(max_preview_rows).to_string())
    else:
        buffer.write("Dataset completo:\n")
        buffer.write(df.to_string())
    buffer.write("\n\n")

    numeric_df = df.select_dtypes(include=[np.number])
    if not numeric_df.empty:
        buffer.write("### Estadísticas descriptivas (basadas en todas las filas)\n")
        buffer.write(numeric_df.describe().to_string())
    else:
        buffer.write("No hay columnas numéricas para mostrar estadísticas.\n")
    buffer.write("\n\n### Valores faltantes\n")
    missing_values = df.isnull().sum()
    missing_df = pd.DataFrame({
        'Columna': missing_values.index,
        'Valores faltantes': missing_values.values,
        'Porcentaje': (missing_values.values / len(df)) * 100
    })
    buffer.write(missing_df.to_string(index=False))
    buffer.write("\n\n### Tipos de datos\n")
    dtypes_df = pd.DataFrame({
        'Columna': df.dtypes.index,
        'Tipo': df.dtypes.values
    })
    buffer.write(dtypes_df.to_string(index=False))
    return buffer.getvalue()

def generate_visualizations(df, target_col=None, cat_col=None):
    plots_info = []

    # Automatically select target and categorical columns if not provided
    if target_col is None:
        target_col = 'Riesgo_Cardiovascular'
    if cat_col is None:
        cat_cols = df.select_dtypes(include=['object', 'category']).columns
        cat_col = cat_cols[0] if len(cat_cols) > 0 else None

    if df.shape[0] < 10:
        print("⚠️ Advertencia: El dataset tiene menos de 10 filas, lo que puede limitar la calidad de las visualizaciones.")

    # 1. Bar plot of target column distribution
    if target_col and target_col in df.columns:
        plt.figure(figsize=(8, 6))
        sns.countplot(x=target_col, data=df, palette='Blues')
        plt.title('Distribución del Riesgo Cardiovascular', fontsize=14)
        plt.xlabel('Riesgo Cardiovascular (0: Bajo, 1: Alto)', fontsize=12)
        plt.ylabel('Frecuencia', fontsize=12)
        plt.tight_layout()
        filename = 'visualizations/dist_target.png'
        plt.savefig(filename)
        plt.close()
        plots_info.append({
            'title': 'Distribución del Riesgo Cardiovascular',
            'filename': filename,
            'type': 'barplot'
        })

    # 2. Bar plot of top categories
    if cat_col and cat_col in df.columns:
        plt.figure(figsize=(10, 6))
        value_counts = df[cat_col].value_counts().head(10)
        if len(value_counts) < 2:
            print(f"⚠️ Advertencia: La categoría seleccionada tiene menos de 2 valores únicos, lo que limita el análisis.")
        sns.barplot(x=value_counts.index, y=value_counts.values, palette='Blues')
        plt.title(f'Conteo de categorías en {cat_col} (Top 10)', fontsize=14)
        plt.xticks(rotation=45, ha='right', fontsize=10)
        plt.ylabel('Cantidad', fontsize=12)
        plt.tight_layout()
        filename = 'visualizations/count_category.png'
        plt.savefig(filename)
        plt.close()
        plots_info.append({
            'title': f'Conteo de categorías en {cat_col} (Top 10)',
            'filename': filename,
            'type': 'barplot'
        })

    # 3. Correlation heatmap
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 1:
        plt.figure(figsize=(10, 8))
        corr_matrix = df[numeric_cols].corr()
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
        sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='Blues', fmt='.2f')
        plt.title('Matriz de Correlación', fontsize=14)
        plt.tight_layout()
        filename = 'visualizations/correlation_matrix.png'
        plt.savefig(filename)
        plt.close()
        plots_info.append({
            'title': 'Matriz de Correlación',
            'filename': filename,
            'type': 'heatmap'
        })

    return plots_info

"""## Predicción de Riesgo Cardiovascular"""
def preprocess_data(df, target_col=None, categorical_cols=None, numerical_cols=None):
    if df.shape[0] < 10:
        raise ValueError("El dataset tiene menos de 10 filas, lo que es insuficiente para entrenar un modelo.")

    # Específico para el dataset conductores_salud
    target_col = 'Riesgo_Cardiovascular'
    categorical_cols = ['Genero', 'Fuma']
    numerical_cols = ['Edad', 'IMC', 'Colesterol', 'Trigliceridos', 'Frecuencia_Cardiaca',
                      'Presion_Sistolica', 'Presion_Diastolica', 'Nivel_Estres', 'Actividad_Fisica']

    # Verificar que las columnas requeridas estén presentes
    required_cols = [target_col] + categorical_cols + numerical_cols
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Faltan las siguientes columnas en el archivo: {', '.join(missing_cols)}")

    # Handle missing values
    df = df.dropna(subset=[target_col])
    for col in numerical_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[col] = df[col].fillna(df[col].median())
    for col in categorical_cols:
        df[col] = df[col].fillna(df[col].mode()[0])

    # Encode categorical variables
    le = LabelEncoder()
    for col in categorical_cols:
        df[col] = le.fit_transform(df[col])

    # Normalize numerical features
    scaler = StandardScaler()
    df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

    return df, scaler, target_col

def train_model(df_encoded, target_col):
    X = df_encoded.drop(columns=[target_col], axis=1)
    y = df_encoded[target_col]

    # Verify numeric data
    non_numeric_cols = X.select_dtypes(exclude=[np.number]).columns
    if len(non_numeric_cols) > 0:
        raise ValueError(f"El conjunto de características contiene columnas no numéricas: {non_numeric_cols.tolist()}")

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Build the Keras model for binary classification
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    # Compile and train
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)

    # Evaluate
    y_pred_prob = model.predict(X_test)
    y_pred = (y_pred_prob > 0.5).astype(int).flatten()

    # Metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Generate visualizations
    plots_info = []

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title('Matriz de Confusión')
    plt.xlabel('Predicho')
    plt.ylabel('Real')
    plt.tight_layout()
    filename = 'visualizations/confusion_matrix.png'
    plt.savefig(filename)
    plt.close()
    plots_info.append({
        'title': 'Matriz de Confusión',
        'filename': filename,
        'type': 'heatmap'
    })

    # Feature Importance (Correlation-based)
    correlations = df_encoded.corr()[target_col].abs().sort_values(ascending=False)
    top_features = correlations[1:6]
    plt.figure(figsize=(10, 6))
    sns.barplot(x=top_features.values, y=top_features.index, palette='Blues')
    plt.title('Importancia de Características (Correlación)', fontsize=14)
    plt.xlabel('Correlación Absoluta', fontsize=12)
    plt.tight_layout()
    filename = 'visualizations/feature_importance.png'
    plt.savefig(filename)
    plt.close()
    plots_info.append({
        'title': 'Importancia de Características',
        'filename': filename,
        'type': 'barplot'
    })

    return accuracy, precision, recall, f1, plots_info, model, X_test, y_test, y_pred, correlations

"""## Integración con Gemini API"""
def query_gemini_analysis(df, query, model, target_col='Riesgo_Cardiovascular', cat_col=None, year_col=None, region_cols=None):
    try:
        if df.shape[0] < 10:
            return "⚠️ Los datos son insuficientes (menos de 10 filas). Por favor, proporciona un dataset más grande para un análisis confiable."

        # Automatically select columns
        if cat_col is None:
            cat_col = 'Genero'
        if year_col is None:
            year_col = None
        if region_cols is None:
            region_cols = []

        # Prepare data summary for Gemini
        df_info = df.head(100).to_string()  # Mostrar las primeras 100 filas como muestra
        columns_info = "\n".join([f"- {col}: {df[col].dtype}" for col in df.columns])
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        stats = "\n".join([f"- {col}: min={df[col].min()}, max={df[col].max()}, mean={df[col].mean()}" for col in numeric_cols])

        # Common instructions for all prompts
        base_instructions = """
        Instrucciones:
        - Responde SOLO a la consulta proporcionada, utilizando únicamente los datos y el análisis proporcionados.
        - NO hagas suposiciones ni generes datos hipotéticos.
        - Proporciona una respuesta concisa, clara y accionable.
        - Si los datos son insuficientes, indica claramente que los datos son limitados y sugiere cómo obtener un análisis más robusto.
        - Evita añadir información o recomendaciones no solicitadas.
        """

        # Specific analysis for predefined questions
        if "highest" in query.lower() and "potential" in query.lower() and cat_col:
            df_encoded, _, _ = preprocess_data(df, target_col, ['Genero', 'Fuma'])
            accuracy, precision, recall, f1, _, model, X_test, y_test, y_pred, _ = train_model(df_encoded, target_col)
            category_risk = df.groupby(cat_col)[target_col].mean().sort_values(ascending=False)
            top_category = category_risk.index[0]
            prompt = f"""
            {base_instructions}

            Datos:
            COLUMNAS:
            {columns_info}

            ESTADÍSTICAS:
            {stats}

            MUESTRA:
            {df_info}

            Análisis: La categoría con mayor riesgo promedio es '{top_category}' con {category_risk.iloc[0]:.2f} (Accuracy del modelo: {accuracy:.2f}). Nota: Los datos tienen {df.shape[0]} filas.
            Confirma este resultado y explica brevemente, en no más de 100 palabras, por qué esta categoría tiene alto riesgo cardiovascular, basándote en el modelo y los datos. Sugiere cómo mejorar el análisis con más datos.
            """
        elif "combination" in query.lower() and "potential" in query.lower():
            cat_cols = ['Genero', 'Fuma']
            df_encoded, _, _ = preprocess_data(df, target_col, cat_cols)
            accuracy, precision, recall, f1, _, model, X_test, y_test, y_pred, _ = train_model(df_encoded, target_col)
            combo_risk = df.groupby(cat_cols)[target_col].mean().sort_values(ascending=False)
            if not combo_risk.empty:
                top_combo = combo_risk.index[0]
                top_risk = combo_risk.iloc[0]
                prompt = f"""
                {base_instructions}

                Datos:
                COLUMNAS:
                {columns_info}

                ESTADÍSTICAS:
                {stats}

                MUESTRA:
                {df_info}

                Análisis: La combinación con mayor riesgo es '{top_combo[0]}'/'{top_combo[1]}' con {top_risk:.2f} (Accuracy del modelo: {accuracy:.2f}). Nota: Los datos tienen {df.shape[0]} filas.
                Confirma este resultado y explica brevemente, en no más de 100 palabras, por qué esta combinación tiene alto riesgo, basándote en el modelo. Sugiere cómo mejorar el análisis con más datos.
                """
            else:
                return "⚠️ Los datos no contienen información suficiente para determinar combinaciones de categorías."
        elif "marketing strategies" in query.lower() and cat_col:
            df_encoded, _, _ = preprocess_data(df, target_col, ['Genero', 'Fuma'])
            accuracy, precision, recall, f1, _, model, X_test, y_test, y_pred, _ = train_model(df_encoded, target_col)
            top_cats = df.groupby(cat_col)[target_col].mean().sort_values(ascending=False).head(2).index.tolist()
            prompt = f"""
            {base_instructions}

            Datos:
            COLUMNAS:
            {columns_info}

            ESTADÍSTICAS:
            {stats}

            MUESTRA:
            {df_info}

            Análisis: Las categorías con mayor riesgo son {', '.join(top_cats)}. Modelo Accuracy: {accuracy:.2f}. Nota: Los datos tienen {df.shape[0]} filas.
            Genera 3 estrategias de prevención específicas y breves (máximo 50 palabras cada una) para reducir el riesgo cardiovascular en estas categorías, basándote en el modelo y los datos. Sugiere cómo mejorar las estrategias con más datos.
            """
        else:
            prompt = f"""
            {base_instructions}

            Datos:
            COLUMNAS:
            {columns_info}

            ESTADÍSTICAS:
            {stats}

            MUESTRA:
            {df_info}

            Consulta: {query}
            Responde únicamente a la consulta, utilizando los datos proporcionados. Proporciona un análisis claro, conciso y accionable en no más de 150 palabras. Si los datos son insuficientes, indícalos claramente y sugiere cómo obtener un análisis más robusto.
            """

        # Query Gemini API
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Error al consultar a Gemini: {str(e)}"

"""## Interfaz de usuario con Gradio"""
def create_gradio_interface():
    df = None
    model = None
    gemini_model = None
    X_test = None
    y_test = None
    y_pred = None
    target_col = 'Riesgo_Cardiovascular'
    categorical_cols = ['Genero', 'Fuma']
    numerical_cols = ['Edad', 'IMC', 'Colesterol', 'Trigliceridos', 'Frecuencia_Cardiaca',
                      'Presion_Sistolica', 'Presion_Diastolica', 'Nivel_Estres', 'Actividad_Fisica']

    def upload_file(file):
        try:
            nonlocal df, target_col, categorical_cols, numerical_cols
            file_extension = os.path.splitext(file.name)[1].lower()
            if file_extension == '.csv':
                df = pd.read_csv(file.name)
            elif file_extension == '.xlsx':
                df = pd.read_excel(file.name, engine='openpyxl')
            else:
                return None, "Formato no soportado. Usa .csv o .xlsx.", "Error", [], [], []

            # Corregir codificación de 'SÃ­' a 'Sí'
            if 'Fuma' in df.columns:
                df['Fuma'] = df['Fuma'].replace('SÃ­', 'Sí')

            stats = generate_csv_stats(df, max_preview_rows=100)
            return (df.head(100).to_html(), stats, f"Archivo {file_extension} cargado correctamente. Total de filas: {df.shape[0]}",
                    [target_col], categorical_cols, numerical_cols)
        except Exception as e:
            return None, f"Error al cargar el archivo: {str(e)}", "Error", [], [], []

    def update_columns(selected_target, selected_categorical, selected_numerical):
        nonlocal target_col, categorical_cols, numerical_cols
        target_col = selected_target
        categorical_cols = selected_categorical
        numerical_cols = selected_numerical
        return f"Configuración actualizada: Objetivo seleccionado, {len(categorical_cols)} categorías, {len(numerical_cols)} variables numéricas"

    def set_api_key(api_key):
        nonlocal gemini_model
        success, model_instance = setup_gemini_api(api_key)
        if success:
            gemini_model = model_instance
            return "API de Gemini configurada correctamente", api_key
        else:
            return "Error al configurar la API de Gemini", ""

    def ask_gemini(query, api_key):
        nonlocal df, gemini_model, target_col, categorical_cols
        if df is None:
            return "Primero debes cargar un archivo CSV o XLSX"
        if not api_key or gemini_model is None:
            return "Primero debes configurar la API de Gemini"
        try:
            cat_col = categorical_cols[0] if categorical_cols else None
            return query_gemini_analysis(df, query, gemini_model, target_col, cat_col)
        except Exception as e:
            return f"Error al consultar a Gemini: {str(e)}"

    def generate_plots():
        nonlocal df, target_col, categorical_cols
        if df is None:
            return "Primero debes cargar un archivo CSV o XLSX", []
        try:
            cat_col = categorical_cols[0] if categorical_cols else None
            plots_info = generate_visualizations(df, target_col, cat_col)
            return "Visualizaciones generadas correctamente", [plot['filename'] for plot in plots_info]
        except Exception as e:
            return f"Error al generar visualizaciones: {str(e)}", []

    def train_and_evaluate():
        nonlocal df, model, X_test, y_test, y_pred, target_col, categorical_cols, numerical_cols
        if df is None:
            return "Primero debes cargar un archivo CSV o XLSX", [], ""
        try:
            df_encoded, _, target_col = preprocess_data(df, target_col, categorical_cols, numerical_cols)
            accuracy, precision, recall, f1, plots_info, model, X_test, y_test, y_pred, _ = train_model(df_encoded, target_col)
            metrics = f"Accuracy: {accuracy:.4f}\nPrecision: {precision:.4f}\nRecall: {recall:.4f}\nF1-Score: {f1:.4f}"
            return metrics, [plot['filename'] for plot in plots_info], "Modelo entrenado correctamente"
        except Exception as e:
            return f"Error al entrenar el modelo: {str(e)}", [], "Error"

    with gr.Blocks(title="Predicción del Riesgo Cardiovascular") as app:
        gr.Markdown("# 📊 Predicción del Riesgo Cardiovascular")
        gr.Markdown("Carga un archivo CSV o XLSX con datos de salud (puede tener múltiples filas, ej. 1000), analiza los datos, predice el riesgo cardiovascular y consulta patrones con Gemini AI.")
        saved_api_key = gr.State("")
        with gr.Tab("Cargar Datos"):
            with gr.Row():
                with gr.Column(scale=1):
                    file_input = gr.File(label="Selecciona un archivo CSV o XLSX", file_types=[".csv", ".xlsx"])
                    upload_button = gr.Button("Cargar Archivo")
                    upload_status = gr.Textbox(label="Estado", interactive=False)
                with gr.Column(scale=2):
                    data_preview = gr.HTML(label="Vista previa (Primeras 100 filas)")
                    data_stats = gr.Textbox(label="Estadísticas (Basadas en todas las filas)", interactive=False, max_lines=30)
        with gr.Tab("Configurar Análisis"):
            target_dropdown = gr.Dropdown(label="Variable Objetivo", choices=['Riesgo_Cardiovascular'], value='Riesgo_Cardiovascular', interactive=True)
            categorical_checklist = gr.CheckboxGroup(label="Variables Categóricas", choices=['Genero', 'Fuma'], value=['Genero', 'Fuma'], interactive=True)
            numerical_checklist = gr.CheckboxGroup(label="Variables Numéricas", choices=['Edad', 'IMC', 'Colesterol', 'Trigliceridos', 'Frecuencia_Cardiaca',
                                                                                       'Presion_Sistolica', 'Presion_Diastolica', 'Nivel_Estres', 'Actividad_Fisica'],
                                                  value=['Edad', 'IMC', 'Colesterol', 'Trigliceridos', 'Frecuencia_Cardiaca',
                                                         'Presion_Sistolica', 'Presion_Diastolica', 'Nivel_Estres', 'Actividad_Fisica'], interactive=True)
            update_button = gr.Button("Actualizar Configuración")
            column_status = gr.Textbox(label="Estado de Configuración", interactive=False)
        with gr.Tab("Configurar API"):
            gemini_api_key = gr.Textbox(label="API Key de Gemini AI", type="password")
            api_button = gr.Button("Configurar API")
            api_status = gr.Textbox(label="Estado", interactive=False)
        with gr.Tab("Consultar Datos"):
            query_input = gr.Textbox(label="Consulta sobre los datos", placeholder="Ej: ¿Qué categoría tiene mayor riesgo cardiovascular?")
            query_button = gr.Button("Consultar a Gemini AI")
            query_result = gr.Markdown(label="Respuesta")
        with gr.Tab("Visualizaciones"):
            viz_button = gr.Button("Generar Visualizaciones")
            viz_status = gr.Textbox(label="Estado", interactive=False)
            viz_gallery = gr.Gallery(label="Gráficos", show_label=True, columns=2, rows=3, height=600)
        with gr.Tab("Predicción"):
            pred_button = gr.Button("Entrenar y Evaluar Modelo")
            pred_metrics = gr.Textbox(label="Métricas del Modelo", interactive=False)
            pred_gallery = gr.Gallery(label="Gráficos de Predicción", show_label=True, columns=2, rows=2, height=600)
            pred_status = gr.Textbox(label="Estado", interactive=False)

        upload_button.click(
            upload_file,
            inputs=[file_input],
            outputs=[data_preview, data_stats, upload_status, target_dropdown, categorical_checklist, numerical_checklist]
        )
        update_button.click(
            update_columns,
            inputs=[target_dropdown, categorical_checklist, numerical_checklist],
            outputs=[column_status]
        )
        api_button.click(set_api_key, inputs=[gemini_api_key], outputs=[api_status, saved_api_key])
        query_button.click(ask_gemini, inputs=[query_input, saved_api_key], outputs=[query_result])
        viz_button.click(generate_plots, inputs=[], outputs=[viz_status, viz_gallery])
        pred_button.click(train_and_evaluate, inputs=[], outputs=[pred_metrics, pred_gallery, pred_status])

    return app

"""## Iniciar la aplicación"""
print("¡Bienvenido al Sistema de Predicción del Riesgo Cardiovascular!")
print("Este notebook te permite:")
print("1. Cargar un archivo CSV o XLSX con datos de conductores (puede tener múltiples filas, ej. 1000)")
print("2. Configurar el análisis seleccionando variables objetivo, categóricas y numéricas")
print("3. Analizar datos (estadísticas y visualizaciones basadas en todas las filas)")
print("4. Predecir el riesgo cardiovascular usando un modelo de red neuronal")
print("5. Consultar patrones y recomendaciones con Gemini AI")
print("\nEjecuta la siguiente celda para iniciar la interfaz.")
app = create_gradio_interface()
app.launch(debug=True)

"""
## Instrucciones de uso

1. **Instalar dependencias**:
   - Ejecuta en Colab:
     ```bash
     !pip install numpy pandas matplotlib seaborn tensorflow google-generativeai gradio scikit-learn openpyxl"""

¡Bienvenido al Sistema de Predicción del Riesgo Cardiovascular!
Este notebook te permite:
1. Cargar un archivo CSV o XLSX con datos de conductores (puede tener múltiples filas, ej. 1000)
2. Configurar el análisis seleccionando variables objetivo, categóricas y numéricas
3. Analizar datos (estadísticas y visualizaciones basadas en todas las filas)
4. Predecir el riesgo cardiovascular usando un modelo de red neuronal
5. Consultar patrones y recomendaciones con Gemini AI

Ejecuta la siguiente celda para iniciar la interfaz.
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b550d23a7ab8f3d723.gradio.live

This share link expires in 1 week. For 

✅ API de Gemini configurada correctamente


/usr/local/lib/python3.11/dist-packages/gradio/components/dropdown.py:227: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: ['Riesgo_Cardiovascular'] or set allow_custom_value=True.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/components/dropdown.py:227: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: ['Riesgo_Cardiovascular'] or set allow_custom_value=True.
  warnings.warn(
<ipython-input-6-694e16ca2a38>:95: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(x=target_col, data=df, palette='Blues')
<ipython-input-6-694e16ca2a38>:115: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` 

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


<ipython-input-6-694e16ca2a38>:243: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=top_features.values, y=top_features.index, palette='Blues')
